In [47]:
import pandas as pd
import numpy as np
import spacy
from string import punctuation
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from collections import Counter

# AI models
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Classifiers
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, StackingClassifier

In [48]:
# Import data
test = pd.read_excel("data\\test.xlsx")
test.head(20)

,Dialog_ID,Msg,Anotador1,Anotador2,SentimentoRegressao,SentimentoClassificacao
0,916,Olá ⁦⁦@VodafonePT⁩ podem-me ajudar ? Tenho a o...,-2.0,-2.0,-2.0,-2
1,916,@nrg78 Olá. Pelo print a subscrição terá sido ...,0.0,0.0,0.0,0
2,917,sou o único com problemas no router da vodafon...,-1.0,-1.0,-1.0,-1
3,917,@bennyys17 Para que a situação seja analisada ...,0.0,0.0,0.0,0
4,918,Menos uma? HORA Bolas! 😱⏰ #MudançaDaHora\nQuem...,-1.0,-1.0,-1.0,-1
5,918,@VodafonePT Óptimo serviço prestado hoje: troc...,1.0,1.0,1.0,1
6,918,@freitasdesousa Olá. Agradecemos o seu feedbac...,1.0,1.0,1.0,1
7,919,"@VodafonePT okaaaaay, onde está a VH1? Já não ...",-1.0,-1.0,-1.0,-1
8,919,@DearVRodrigues Olá @DearVRodrigues. Informamo...,0.0,0.0,0.0,0
9,920,"@VodafonePT, não sei que raio fazem aos cartõe...",-2.0,-2.0,-2.0,-2


In [49]:
train = pd.read_excel("data\\train.xlsx")
train.head(10)

,Dialog_ID,SentimentoRegressao,SentimentoClassificacao,Msg
0,0,-1.0,-1,"@nowoportugal alô, preciso de internet pelo me..."
1,0,0.0,0,Olá @Paulozxl lamentamos a situação que nos re...
2,1,-1.5,-2,@nowoportugal Posso terminar o contrato e muda...
3,1,0.0,0,Olá @mpnm77 lamentamos a situação que se sinta...
4,2,-1.0,-1,"@nowoportugal sem sinal de Nowo, tampoco net"
5,2,0.0,0,"Olá @doradarocha, lamentamos a situação, mas e..."
6,3,-1.0,-1,@nowoportugal têm o serviço de net em baixo e ...
7,3,0.0,0,"Olá @pngantunes, lamentamos a situação, mas ex..."
8,4,0.5,0,Hoje é dia de ver o último concerto de Simone ...
9,4,0.5,1,@IuriPereira09 @RTP1 Certo! Aqui está: \n\nhtt...


In [50]:
nan_counts = train.isna().sum()
print(nan_counts)
print("----------------------")
nan_counts = test.isna().sum()
print(nan_counts)

Dialog_ID                  0
SentimentoRegressao        0
SentimentoClassificacao    0
Msg                        0
dtype: int64
----------------------
Dialog_ID                  0
Msg                        0
Anotador1                  6
Anotador2                  2
SentimentoRegressao        0
SentimentoClassificacao    0
dtype: int64


In [51]:
y_train = train["SentimentoClassificacao"].values
x_train = train["Msg"].values

y_test = test["SentimentoClassificacao"].values
x_test = test["Msg"].values

vectorizer = TfidfVectorizer(max_features=100)
x_train = vectorizer.fit_transform(x_train.astype(str))
x_test = vectorizer.transform(x_test.astype(str))
print(x_test)

  (0, 98)	0.22200704338526753
  (0, 94)	0.21426175453144208
  (0, 92)	0.30426052698217
  (0, 74)	0.14211943916623965
  (0, 60)	0.22524567358086073
  (0, 57)	0.5739113220646512
  (0, 47)	0.2954461516525767
  (0, 46)	0.28287126666134277
  (0, 37)	0.15589837836664536
  (0, 27)	0.2838858993425227
  (0, 22)	0.13617496334821658
  (0, 13)	0.15589837836664536
  (0, 2)	0.3112467439687426
  (1, 97)	0.5047534571180281
  (1, 95)	0.3683074939186297
  (1, 91)	0.2687068660382961
  (1, 83)	0.2300630353154174
  (1, 81)	0.20679009536276383
  (1, 74)	0.11496702193013539
  (1, 67)	0.2620654530442102
  (1, 63)	0.13494016480614304
  (1, 60)	0.1822116977533814
  (1, 57)	0.15475451713207872
  (1, 56)	0.2079407429615536
  (1, 55)	0.38519910718910216
  :	:
  (1902, 28)	0.17536773044914658
  (1902, 26)	0.2019932177225537
  (1902, 22)	0.13117246651785336
  (1902, 20)	0.31546801052070456
  (1902, 19)	0.3803837350059967
  (1902, 14)	0.1934215028242839
  (1902, 13)	0.15017132601823638
  (1903, 84)	0.2485578960094241

In [52]:
weights = ['uniform', 'distance']
for k in range(1, 11):
    for w in weights:
        clf = KNeighborsClassifier(n_neighbors=k, weights=w)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
        print(f"Accuracy of KNN for k={k} and w={w}: {accuracy}%")
    print("")

print("Best hyperparameters: k=9 & w=distance.")

Accuracy of KNN for k=1 and w=uniform: 41.89%
Accuracy of KNN for k=1 and w=distance: 41.89%

Accuracy of KNN for k=2 and w=uniform: 43.41%
Accuracy of KNN for k=2 and w=distance: 43.31%

Accuracy of KNN for k=3 and w=uniform: 45.04%
Accuracy of KNN for k=3 and w=distance: 45.35%

Accuracy of KNN for k=4 and w=uniform: 46.77%
Accuracy of KNN for k=4 and w=distance: 48.35%

Accuracy of KNN for k=5 and w=uniform: 47.77%
Accuracy of KNN for k=5 and w=distance: 48.14%

Accuracy of KNN for k=6 and w=uniform: 49.19%
Accuracy of KNN for k=6 and w=distance: 49.76%

Accuracy of KNN for k=7 and w=uniform: 49.13%
Accuracy of KNN for k=7 and w=distance: 50.03%

Accuracy of KNN for k=8 and w=uniform: 49.13%
Accuracy of KNN for k=8 and w=distance: 49.97%

Accuracy of KNN for k=9 and w=uniform: 48.71%
Accuracy of KNN for k=9 and w=distance: 50.81%

Accuracy of KNN for k=10 and w=uniform: 49.19%
Accuracy of KNN for k=10 and w=distance: 50.24%

Best hyperparameters: k=9 & w=distance.


In [53]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
for c in range(1, 11):
    for k in kernels:
        clf = SVC(C=c, kernel=k)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
        print(f"Accuracy of SVM for c={c} and kernel={k}: {accuracy}%")
    print("")

print("Best hyperparameters: c=1 & kernel=rbf.")

Accuracy of SVM for c=1 and kernel=linear: 51.65%
Accuracy of SVM for c=1 and kernel=poly: 51.97%
Accuracy of SVM for c=1 and kernel=rbf: 52.49%
Accuracy of SVM for c=1 and kernel=sigmoid: 46.46%

Accuracy of SVM for c=2 and kernel=linear: 49.55%
Accuracy of SVM for c=2 and kernel=poly: 50.29%
Accuracy of SVM for c=2 and kernel=rbf: 51.97%
Accuracy of SVM for c=2 and kernel=sigmoid: 45.77%

Accuracy of SVM for c=3 and kernel=linear: 48.77%
Accuracy of SVM for c=3 and kernel=poly: 49.19%
Accuracy of SVM for c=3 and kernel=rbf: 50.92%
Accuracy of SVM for c=3 and kernel=sigmoid: 43.31%

Accuracy of SVM for c=4 and kernel=linear: 48.03%
Accuracy of SVM for c=4 and kernel=poly: 49.29%
Accuracy of SVM for c=4 and kernel=rbf: 50.55%
Accuracy of SVM for c=4 and kernel=sigmoid: 44.88%

Accuracy of SVM for c=5 and kernel=linear: 47.87%
Accuracy of SVM for c=5 and kernel=poly: 49.24%
Accuracy of SVM for c=5 and kernel=rbf: 49.34%
Accuracy of SVM for c=5 and kernel=sigmoid: 41.78%

Accuracy of SVM

In [54]:
criterions = ['gini', 'entropy', 'log_loss']

for criterion in criterions:
	for depth in range(1, 11):
		tree = DecisionTreeClassifier(criterion=criterion, max_depth=depth)
		tree.fit(x_train, y_train)
		y_pred = tree.predict(x_test)
		accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
		print(f"Accuracy for criterion={criterion} and depth={depth}: {accuracy}%")
	print("")

print("Best hyperparameters: criterion=log_loss & depth=3.")

Accuracy for criterion=gini and depth=1: 49.4%
Accuracy for criterion=gini and depth=2: 49.61%
Accuracy for criterion=gini and depth=3: 47.93%
Accuracy for criterion=gini and depth=4: 46.88%
Accuracy for criterion=gini and depth=5: 52.02%
Accuracy for criterion=gini and depth=6: 52.49%
Accuracy for criterion=gini and depth=7: 52.13%
Accuracy for criterion=gini and depth=8: 50.92%
Accuracy for criterion=gini and depth=9: 49.92%
Accuracy for criterion=gini and depth=10: 51.39%

Accuracy for criterion=entropy and depth=1: 49.4%
Accuracy for criterion=entropy and depth=2: 49.76%
Accuracy for criterion=entropy and depth=3: 55.7%
Accuracy for criterion=entropy and depth=4: 52.39%
Accuracy for criterion=entropy and depth=5: 52.49%
Accuracy for criterion=entropy and depth=6: 53.12%
Accuracy for criterion=entropy and depth=7: 53.07%
Accuracy for criterion=entropy and depth=8: 51.08%
Accuracy for criterion=entropy and depth=9: 51.39%
Accuracy for criterion=entropy and depth=10: 50.6%

Accuracy f

In [55]:
# Random Forest
random_forest = RandomForestClassifier()
random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)
accuracy_rf = round(accuracy_score(y_test, y_pred) * 100, 2)
print(f"Accuracy for Random Forest: {accuracy}%")
print(classification_report(y_test, y_pred))

# Bagging Classifier
bagging = BaggingClassifier()
bagging.fit(x_train, y_train)
y_pred_bagging = bagging.predict(x_test)
accuracy_bagging = round(accuracy_score(y_test, y_pred_bagging) * 100, 2)
print(f"Accuracy for Bagging: {accuracy_bagging}%")
print(classification_report(y_test, y_pred_bagging))

Accuracy for Random Forest: 51.02%
              precision    recall  f1-score   support

          -2       0.66      0.09      0.16       292
          -1       0.36      0.39      0.37       449
           0       0.58      0.81      0.68       936
           1       0.34      0.12      0.18       228

    accuracy                           0.52      1905
   macro avg       0.48      0.35      0.35      1905
weighted avg       0.51      0.52      0.47      1905

Accuracy for Bagging: 48.56%
              precision    recall  f1-score   support

          -2       0.52      0.11      0.18       292
          -1       0.36      0.41      0.38       449
           0       0.56      0.72      0.63       936
           1       0.25      0.14      0.18       228

    accuracy                           0.49      1905
   macro avg       0.42      0.35      0.34      1905
weighted avg       0.47      0.49      0.45      1905



In [56]:
logistic = LogisticRegression()
logistic.fit(x_train, y_train)
y_pred = logistic.predict(x_test)
accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
print(f"Accuracy for Logistic: {accuracy}%")
print(classification_report(y_test, y_pred))

perceptron = Perceptron()
perceptron.fit(x_train, y_train)
y_pred = perceptron.predict(x_test)
accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
print(f"Accuracy for Perceptron: {accuracy}%")
print(classification_report(y_test, y_pred))

Accuracy for Logistic: 51.39%
              precision    recall  f1-score   support

          -2       0.41      0.03      0.06       292
          -1       0.34      0.36      0.35       449
           0       0.58      0.84      0.68       936
           1       0.54      0.09      0.16       228

    accuracy                           0.51      1905
   macro avg       0.47      0.33      0.31      1905
weighted avg       0.49      0.51      0.45      1905

Accuracy for Perceptron: 34.65%
              precision    recall  f1-score   support

          -2       0.27      0.08      0.12       292
          -1       0.23      0.30      0.26       449
           0       0.53      0.50      0.51       936
           1       0.10      0.14      0.12       228

    accuracy                           0.35      1905
   macro avg       0.28      0.26      0.25      1905
weighted avg       0.36      0.35      0.35      1905



In [57]:
# TEST THE HYPERPARAMETERS
knn = KNeighborsClassifier(n_neighbors=9, weights="distance")
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
print(f"Accuracy of KNN: {accuracy}%")
print(classification_report(y_test, y_pred))

svm = SVC(C=1, kernel="rbf")
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
print(f"Accuracy of SVM: {accuracy}%")
print(classification_report(y_test, y_pred))

tree = DecisionTreeClassifier(criterion="log_loss", max_depth=3)
tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)
accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
print(f"Accuracy of tree: {accuracy}%")
print(classification_report(y_test, y_pred))

bernoulli = BernoulliNB()
bernoulli.fit(x_train, y_train)
y_pred = bernoulli.predict(x_test)
accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
print(f"Accuracy of bernoulliNB: {accuracy}%")
print(classification_report(y_test, y_pred))

multi = MultinomialNB()
multi.fit(x_train, y_train)
y_pred = multi.predict(x_test)
accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
print(f"Accuracy of multinomialNB: {accuracy}%")
print(classification_report(y_test, y_pred))

Accuracy of KNN: 50.6%
              precision    recall  f1-score   support

          -2       0.56      0.15      0.24       292
          -1       0.35      0.41      0.37       449
           0       0.58      0.74      0.66       936
           1       0.36      0.18      0.24       228

    accuracy                           0.51      1905
   macro avg       0.46      0.37      0.38      1905
weighted avg       0.50      0.51      0.48      1905

Accuracy of SVM: 52.49%
              precision    recall  f1-score   support

          -2       0.88      0.02      0.05       292
          -1       0.36      0.41      0.38       449
           0       0.58      0.85      0.69       936
           1       0.58      0.07      0.12       228

    accuracy                           0.52      1905
   macro avg       0.60      0.34      0.31      1905
weighted avg       0.57      0.52      0.45      1905

Accuracy of tree: 55.7%
              precision    recall  f1-score   support

    

C:\Users\valko\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\valko\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\valko\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [ ]:
# Stacking Classifier (usando Logistic Regression y Random Forest como modelos base)
stacking = StackingClassifier(
    estimators=[
        ('logistic', LogisticRegression()),		# 51.39%
        ('tree', DecisionTreeClassifier()),	# 55.7%
        ('svm', SVC(C=1, kernel="rbf")),		# 52.49%
        ('knn', KNeighborsClassifier(n_neighbors=9, weights="distance")),	# 50.6%
		('Random forest', RandomForestClassifier()),	# 50.87%
        ("Bernoulli", MultinomialNB())		# 50.71%
    ],
    final_estimator=SVC(C=1, kernel="rbf")
)
stacking.fit(x_train, y_train)
y_pred_stacking = stacking.predict(x_test)
accuracy_stacking = round(accuracy_score(y_test, y_pred_stacking) * 100, 2)
print(f"Accuracy for Stacking: {accuracy_stacking}%")
print(classification_report(y_test, y_pred_stacking))

Accuracy for Stacking: 53.23%
              precision    recall  f1-score   support

          -2       0.00      0.00      0.00       292
          -1       0.40      0.34      0.37       449
           0       0.56      0.90      0.69       936
           1       0.68      0.07      0.12       228

    accuracy                           0.53      1905
   macro avg       0.41      0.33      0.30      1905
weighted avg       0.45      0.53      0.44      1905



C:\Users\valko\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\valko\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\valko\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [59]:
x_train_dense = x_train.toarray() if hasattr(x_train, "toarray") else x_train
y_train_dense = y_train.toarray() if hasattr(y_train, "toarray") else y_train
x_test_dense = x_test.toarray() if hasattr(x_test, "toarray") else x_test
y_test_dense = y_test.toarray() if hasattr(y_test, "toarray") else y_test

gauss = GaussianNB()
gauss.fit(x_train_dense, y_train_dense)
y_pred = gauss.predict(x_test_dense)
accuracy = round(accuracy_score(y_test_dense, y_pred)*100, 2)
print(f"Accuracy of gaussianNB: {accuracy}%")
print(classification_report(y_test, y_pred))

Accuracy of gaussianNB: 22.78%
              precision    recall  f1-score   support

          -2       0.18      0.86      0.30       292
          -1       0.21      0.01      0.02       449
           0       0.72      0.08      0.14       936
           1       0.27      0.48      0.34       228

    accuracy                           0.23      1905
   macro avg       0.35      0.35      0.20      1905
weighted avg       0.47      0.23      0.16      1905



As we can see the best classfier is the Decision Tree with a 55.7% of accuracy